In [ ]:
#r "nuget: Neo4j.Driver, 5.17"
#r "nuget: Microsoft.SemanticKernel, 1.5"
#r "nuget: Microsoft.Extensions.Configuration, 8.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0"

using Neo4j.Driver;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.ComponentModel;

In [ ]:
var config = new ConfigurationBuilder().AddUserSecrets("2a78a9a7-426d-457c-b4d0-42438d70fa71").Build();

In [ ]:
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion("gpt-3.5-turbo", config["OAI:ApiKey"]);
var kernel = builder.Build();

In [ ]:
var chatService = kernel.GetRequiredService<IChatCompletionService>();
var promptSettings = new OpenAIPromptExecutionSettings
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};
var chatHistory = new ChatHistory();

In [ ]:
var chat = new Func<string, Task>(async userInput =>
{
    if (string.IsNullOrEmpty(userInput))
        return;

    chatHistory.AddUserMessage(userInput);
    var response = await chatService.GetChatMessageContentAsync(chatHistory, promptSettings, kernel);
    chatHistory.AddMessage(response.Role, response.Content ?? string.Empty);
    Console.WriteLine(response);
});

In [ ]:
await chat("add message here");